"""
DATA CLEANING - Nettoyage des données
======================================

Objectifs :
1. Analyser les valeurs manquantes
2. Filtrer les utilisateurs et films peu actifs
3. Assurer la qualité des données pour le modèle
4. Sauvegarder les données nettoyées
"""

In [1]:

# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery

# Configuration des graphiques
%matplotlib inline
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Bibliothèques importées")

✅ Bibliothèques importées


In [2]:
# Connexion BigQuery
project_id = 'students-group1'
client = bigquery.Client(project=project_id)

print("📥 Chargement des données brutes depuis BigQuery...")
print("="*80)

# Charger les films
query_movies = "SELECT * FROM `master-ai-cloud.MoviePlatform.movies`"
df_movies_raw = client.query(query_movies).to_dataframe()

# Charger TOUTES les ratings
query_ratings = "SELECT * FROM `master-ai-cloud.MoviePlatform.ratings`"
df_ratings_raw = client.query(query_ratings).to_dataframe()

print(f"✅ Films bruts    : {len(df_movies_raw)} lignes")
print(f"✅ Ratings bruts  : {len(df_ratings_raw)} lignes")
print(f"✅ Utilisateurs   : {df_ratings_raw['userId'].nunique()} uniques")

📥 Chargement des données brutes depuis BigQuery...
✅ Films bruts    : 10329 lignes
✅ Ratings bruts  : 105339 lignes
✅ Utilisateurs   : 668 uniques


In [3]:
print("="*80)
print("🔍 ANALYSE DES VALEURS MANQUANTES")
print("="*80)

# Table MOVIES
print("\n📊 Table MOVIES :")
print("-" * 40)
missing_movies = df_movies_raw.isnull().sum()
print(missing_movies)
print(f"\nTotal valeurs manquantes : {missing_movies.sum()}")

# Genres spéciaux
no_genres = (df_movies_raw['genres'] == "(no genres listed)").sum()
print(f"Films sans genre listé : {no_genres} ({no_genres/len(df_movies_raw)*100:.1f}%)")

# Table RATINGS
print("\n📊 Table RATINGS :")
print("-" * 40)
missing_ratings = df_ratings_raw.isnull().sum()
print(missing_ratings)
print(f"\nTotal valeurs manquantes : {missing_ratings.sum()}")

print("\n✅ Analyse des valeurs manquantes terminée")

🔍 ANALYSE DES VALEURS MANQUANTES

📊 Table MOVIES :
----------------------------------------
movieId    0
title      0
genres     0
dtype: int64

Total valeurs manquantes : 0
Films sans genre listé : 7 (0.1%)

📊 Table RATINGS :
----------------------------------------
userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

Total valeurs manquantes : 0

✅ Analyse des valeurs manquantes terminée


In [4]:
print("="*80)
print("📊 ANALYSE DE LA DISTRIBUTION")
print("="*80)

# Activité des utilisateurs
user_activity = df_ratings_raw.groupby('userId').size()
print("\n👤 UTILISATEURS :")
print(f"  - Total : {len(user_activity)}")
print(f"  - Moyenne de notes par utilisateur : {user_activity.mean():.1f}")
print(f"  - Médiane : {user_activity.median():.0f}")
print(f"  - Min : {user_activity.min()}")
print(f"  - Max : {user_activity.max()}")
print(f"  - Utilisateurs avec < 5 notes : {(user_activity < 5).sum()} ({(user_activity < 5).sum()/len(user_activity)*100:.1f}%)")
print(f"  - Utilisateurs avec < 10 notes : {(user_activity < 10).sum()} ({(user_activity < 10).sum()/len(user_activity)*100:.1f}%)")

# Popularité des films
movie_popularity = df_ratings_raw.groupby('movieId').size()
print("\n🎬 FILMS :")
print(f"  - Films notés : {len(movie_popularity)}")
print(f"  - Films dans la base : {len(df_movies_raw)}")
print(f"  - Films jamais notés : {len(df_movies_raw) - len(movie_popularity)}")
print(f"  - Moyenne de notes par film : {movie_popularity.mean():.1f}")
print(f"  - Médiane : {movie_popularity.median():.0f}")
print(f"  - Films avec < 5 notes : {(movie_popularity < 5).sum()} ({(movie_popularity < 5).sum()/len(movie_popularity)*100:.1f}%)")
print(f"  - Films avec < 10 notes : {(movie_popularity < 10).sum()} ({(movie_popularity < 10).sum()/len(movie_popularity)*100:.1f}%)")

# Distribution des notes
print("\n⭐ NOTES :")
print(df_ratings_raw['rating'].describe())

📊 ANALYSE DE LA DISTRIBUTION

👤 UTILISATEURS :
  - Total : 668
  - Moyenne de notes par utilisateur : 157.7
  - Médiane : 70
  - Min : 20
  - Max : 5678
  - Utilisateurs avec < 5 notes : 0 (0.0%)
  - Utilisateurs avec < 10 notes : 0 (0.0%)

🎬 FILMS :
  - Films notés : 10325
  - Films dans la base : 10329
  - Films jamais notés : 4
  - Moyenne de notes par film : 10.2
  - Médiane : 3
  - Films avec < 5 notes : 6470 (62.7%)
  - Films avec < 10 notes : 8010 (77.6%)

⭐ NOTES :
count    105339.000000
mean          3.516850
std           1.044872
min           0.500000
25%           3.000000
50%           3.500000
75%           4.000000
max           5.000000
Name: rating, dtype: float64


"""netooyage des donne""" 

In [5]:
print("="*80)
print("🔧 APPLICATION DU NETTOYAGE")
print("="*80)

# Copier les données
df_movies_clean = df_movies_raw.copy()
df_ratings_clean = df_ratings_raw.copy()

print("\n📊 AVANT nettoyage :")
print(f"  - Films : {len(df_movies_clean)}")
print(f"  - Ratings : {len(df_ratings_clean)}")
print(f"  - Utilisateurs : {df_ratings_clean['userId'].nunique()}")

# 1. Supprimer les films sans genre
df_movies_clean = df_movies_clean[df_movies_clean['genres'] != "(no genres listed)"]
print(f"\n✂️ Étape 1 : Films sans genre supprimés")
print(f"  - Films restants : {len(df_movies_clean)}")

# 2. Garder seulement les ratings des films restants
df_ratings_clean = df_ratings_clean[df_ratings_clean['movieId'].isin(df_movies_clean['movieId'])]
print(f"\n✂️ Étape 2 : Ratings des films supprimés retirés")
print(f"  - Ratings restants : {len(df_ratings_clean)}")

# 3. Filtrer les films avec < 5 notes
movie_counts = df_ratings_clean.groupby('movieId').size()
valid_movies = movie_counts[movie_counts >= 5].index
df_movies_clean = df_movies_clean[df_movies_clean['movieId'].isin(valid_movies)]
df_ratings_clean = df_ratings_clean[df_ratings_clean['movieId'].isin(valid_movies)]
print(f"\n✂️ Étape 3 : Films avec < 5 notes supprimés")
print(f"  - Films restants : {len(df_movies_clean)}")
print(f"  - Ratings restants : {len(df_ratings_clean)}")

# 4. Filtrer les utilisateurs avec < 10 notes
user_counts = df_ratings_clean.groupby('userId').size()
valid_users = user_counts[user_counts >= 10].index
df_ratings_clean = df_ratings_clean[df_ratings_clean['userId'].isin(valid_users)]
print(f"\n✂️ Étape 4 : Utilisateurs avec < 10 notes supprimés")
print(f"  - Utilisateurs restants : {df_ratings_clean['userId'].nunique()}")
print(f"  - Ratings restants : {len(df_ratings_clean)}")

# 5. Re-filtrer les films (certains peuvent avoir perdu des notes)
movie_counts_final = df_ratings_clean.groupby('movieId').size()
valid_movies_final = movie_counts_final[movie_counts_final >= 5].index
df_movies_clean = df_movies_clean[df_movies_clean['movieId'].isin(valid_movies_final)]
df_ratings_clean = df_ratings_clean[df_ratings_clean['movieId'].isin(valid_movies_final)]

print(f"\n✂️ Étape 5 : Nettoyage final")
print(f"  - Films finaux : {len(df_movies_clean)}")
print(f"  - Ratings finaux : {len(df_ratings_clean)}")
print(f"  - Utilisateurs finaux : {df_ratings_clean['userId'].nunique()}")

print("\n" + "="*80)
print("📊 RÉSUMÉ DU NETTOYAGE")
print("="*80)
print(f"Films : {len(df_movies_raw)} → {len(df_movies_clean)} ({len(df_movies_clean)/len(df_movies_raw)*100:.1f}%)")
print(f"Ratings : {len(df_ratings_raw)} → {len(df_ratings_clean)} ({len(df_ratings_clean)/len(df_ratings_raw)*100:.1f}%)")
print(f"Utilisateurs : {df_ratings_raw['userId'].nunique()} → {df_ratings_clean['userId'].nunique()} ({df_ratings_clean['userId'].nunique()/df_ratings_raw['userId'].nunique()*100:.1f}%)")
print("="*80)

🔧 APPLICATION DU NETTOYAGE

📊 AVANT nettoyage :
  - Films : 10329
  - Ratings : 105339
  - Utilisateurs : 668

✂️ Étape 1 : Films sans genre supprimés
  - Films restants : 10322

✂️ Étape 2 : Ratings des films supprimés retirés
  - Ratings restants : 105332

✂️ Étape 3 : Films avec < 5 notes supprimés
  - Films restants : 3855
  - Ratings restants : 94121

✂️ Étape 4 : Utilisateurs avec < 10 notes supprimés
  - Utilisateurs restants : 668
  - Ratings restants : 94121

✂️ Étape 5 : Nettoyage final
  - Films finaux : 3855
  - Ratings finaux : 94121
  - Utilisateurs finaux : 668

📊 RÉSUMÉ DU NETTOYAGE
Films : 10329 → 3855 (37.3%)
Ratings : 105339 → 94121 (89.4%)
Utilisateurs : 668 → 668 (100.0%)


In [6]:
print("="*80)
print("☁️ SAUVEGARDE DANS BIGQUERY")
print("="*80)

# Configuration
project_id = 'students-group1'
dataset_id = 'group1_movie_analysis'

print(f"\n📤 Upload vers BigQuery : {project_id}.{dataset_id}")
print("⏳ Cela peut prendre 1-2 minutes...\n")

# Table 1 : Movies nettoyés
table_id_movies = f'{project_id}.{dataset_id}.movies_clean'
print(f"1️⃣ Création de la table : {table_id_movies}")

# Préparer les données (supprimer la colonne genres_list qui n'est pas sérialisable)
df_movies_to_upload = df_movies_clean[['movieId', 'title', 'genres']].copy()

# Upload vers BigQuery
df_movies_to_upload.to_gbq(
    destination_table=f'{dataset_id}.movies_clean',
    project_id=project_id,
    if_exists='replace',  # Remplace si existe déjà
    progress_bar=False
)
print(f"   ✅ {len(df_movies_to_upload)} films uploadés")

# Table 2 : Ratings nettoyés
table_id_ratings = f'{project_id}.{dataset_id}.ratings_clean'
print(f"\n2️⃣ Création de la table : {table_id_ratings}")

df_ratings_clean.to_gbq(
    destination_table=f'{dataset_id}.ratings_clean',
    project_id=project_id,
    if_exists='replace',
    progress_bar=False
)
print(f"   ✅ {len(df_ratings_clean)} ratings uploadés")

print("\n" + "="*80)
print("🎉 DONNÉES SAUVEGARDÉES DANS BIGQUERY AVEC SUCCÈS !")
print("="*80)
print(f"\nVous pouvez maintenant requêter :")
print(f"  SELECT * FROM `{table_id_movies}` LIMIT 10")
print(f"  SELECT * FROM `{table_id_ratings}` LIMIT 10")

☁️ SAUVEGARDE DANS BIGQUERY

📤 Upload vers BigQuery : students-group1.group1_movie_analysis
⏳ Cela peut prendre 1-2 minutes...

1️⃣ Création de la table : students-group1.group1_movie_analysis.movies_clean
   ✅ 3855 films uploadés

2️⃣ Création de la table : students-group1.group1_movie_analysis.ratings_clean
   ✅ 94121 ratings uploadés

🎉 DONNÉES SAUVEGARDÉES DANS BIGQUERY AVEC SUCCÈS !

Vous pouvez maintenant requêter :
  SELECT * FROM `students-group1.group1_movie_analysis.movies_clean` LIMIT 10
  SELECT * FROM `students-group1.group1_movie_analysis.ratings_clean` LIMIT 10
